# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## 0. 回顾numpy的级联

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

In [ ]:
np.concatenate([n1,n2])

为方便讲解，我们首先定义一个生成DataFrame的函数：

In [14]:
def make_df(inds,cols):
    #字典的key作为列名进行展示
    dic = {key:[key+str(i) for i in inds]for key in cols}
    
    df = DataFrame(dic,index = inds)
    return df

In [6]:
make_df([1,2],list('AB'))

,A,B
0,A1,B1
1,A2,B2


## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

### 1)  简单级联

和np.concatenate一样，优先增加行数（默认axis=0）

In [7]:
df1 = make_df([1,2],list('AB'))
df2 = make_df([3,4],list('AB'))

In [8]:
display(df1,df2)

,A,B
0,A1,B1
1,A2,B2


,A,B
0,A3,B3
1,A4,B4


In [9]:
pd.concat([df1,df2])

,A,B
0,A1,B1
1,A2,B2
0,A3,B3
1,A4,B4


In [11]:
pd.concat((df1,df2),axis = 1)

,A,B,A,B
0,A1,B1,A3,B3
1,A2,B2,A4,B4


In [12]:
pd.concat((df1,df2),ignore_index=True)

,A,B
0,A1,B1
1,A2,B2
2,A3,B3
3,A4,B4


可以通过设置axis来改变级联方向

注意index在级联时可以重复

也可以选择忽略ignore_index，重新索引

或者使用多层索引 keys  

concat([x,y],keys=['x','y'])

In [13]:
pd.concat([df1,df2],keys=['x','y'])

A   B
x 0  A1  B1
  1  A2  B2
y 0  A3  B3
  1  A4  B4

In [ ]:
#pd 模块 import pandas as pd

#df1,df2 具体的实例
#级联的方法，属于上一级，DataFrame来自pandas

============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

In [15]:
df1 = make_df([1,2,3],list('AB'))
df2 = make_df([3,4],list('BCD'))
display(df1,df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [16]:
pd.concat([df1,df2])

,A,B,C,D
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


In [19]:
pd.concat((df1,df2),join = 'inner',axis = 1)

,A,B,B,C,D
3,A3,B3,B3,C3,D3


有3种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

- 连接指定轴 join_axes

In [25]:
df2.columns

Index(['B', 'C', 'D'], dtype='object')

In [24]:
pd.concat([df1,df2],join_axes=[df2.columns])

,B,C,D
1,B1,NaN,NaN
2,B2,NaN,NaN
3,B3,NaN,NaN
3,B3,C3,D3
4,B4,C4,D4


============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [26]:
display(df1,df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [28]:
#append函数属于DataFrame，concat这函数属于pandas模块
pd.concat((df1,df2))
# df1.append(df2)

,A,B,C,D
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

###  1) 一对一合并

In [35]:
#merge根据相同的元素进行合并的
df1 = DataFrame({'name':['张三','李四','Chales'],'id':[1,2,3],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'id':[2,3,4],'group':['sale','search','service']})

# pd.concat((df1,df2))

In [36]:
df1.merge(df2)

,age,id,name,group,sex
0,21,2,李四,sale,男
1,25,3,Chales,search,男


### 2) 多对一合并

In [37]:
#多对一的合并
df1 = DataFrame({'name':['张三','李四','Chales'],'id':[1,2,2],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'id':[2,3,4],'group':['sale','search','service']})
display(df1,df2)

,age,id,name
0,22,1,张三
1,21,2,李四
2,25,2,Chales


,group,id,sex
0,sale,2,男
1,search,3,男
2,service,4,女


In [38]:
df1.merge(df2)

,age,id,name,group,sex
0,21,2,李四,sale,男
1,25,2,Chales,sale,男


### 3) 多对多合并

In [39]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'name':['张三','张三','凡凡'],'group':['sale','search','service']})

In [40]:
#多对多合并乘法的
df1.merge(df2)

,age,name,salary,group,sex
0,22,张三,10000,sale,男
1,22,张三,10000,search,男
2,25,张三,20000,sale,男
3,25,张三,20000,search,男


### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [43]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'age':[21,18,29],'name':['张三','张三','凡凡'],'group':['sale','search','service']})

In [47]:
df1.merge(df2,on = 'name',suffixes=['_a','_b'])

,age_a,name,salary,age_b,group
0,22,张三,10000,21,sale
1,22,张三,10000,18,search
2,25,张三,20000,21,sale
3,25,张三,20000,18,search


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [48]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'年龄':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']})

In [50]:
#No common columns to perform merge on
df1.merge(df2,left_on='name',right_on='名字')

,age,name,salary,group,名字,年龄
0,22,张三,10000,sale,张三,21
1,22,张三,10000,search,张三,18
2,25,张三,20000,sale,张三,21
3,25,张三,20000,search,张三,18


In [51]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'年龄':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']},
                index = [22,21,25])
display(df1,df2)

,age,name,salary
0,22,张三,10000
1,21,李四,12000
2,25,张三,20000


,group,名字,年龄
22,sale,张三,21
21,search,张三,18
25,service,凡凡,29


In [53]:
df1.merge(df2,left_on='age',right_index=True)

,age,name,salary,group,名字,年龄
0,22,张三,10000,sale,张三,21
1,21,李四,12000,search,张三,18
2,25,张三,20000,service,凡凡,29


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

In [54]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'age':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']})

In [55]:
df1.merge(df2,how='inner')

,age,name,salary,group,名字
0,21,李四,12000,sale,张三


In [56]:
df1.merge(df2,how = 'outer')

,age,name,salary,group,名字
0,22,张三,10000.0,NaN,NaN
1,21,李四,12000.0,sale,张三
2,25,张三,20000.0,NaN,NaN
3,18,NaN,NaN,search,张三
4,29,NaN,NaN,service,凡凡


- 外合并 how='outer'：补NaN

- 左合并、右合并：how='left'，how='right'，

In [57]:
df1.merge(df2,how = 'left')

,age,name,salary,group,名字
0,22,张三,10000,NaN,NaN
1,21,李四,12000,sale,张三
2,25,张三,20000,NaN,NaN


============================================

练习17：

1. 如果只有张三赵小六语数英三个科目的成绩，如何合并？

3. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

In [58]:
#期中
df1 = DataFrame({'name':['张三','李四','张三'],'degree':[120,118,149],'age':[22,21,25]})

#期末考试
df2 = DataFrame({'degree':[99,97,129],'name':['张三','张三','凡凡'],'group':['sale','search','service']})

In [60]:
df1.merge(df2,on = 'name',suffixes=['_期中','_期末'])

,age,degree_期中,name,degree_期末,group
0,22,120,张三,99,sale
1,22,120,张三,97,search
2,25,149,张三,99,sale
3,25,149,张三,97,search


============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

首先导入文件，并查看数据样本

In [62]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [63]:
#使用pandas读取数据
pop = pd.read_csv('../../data/state-population.csv')

areas = pd.read_csv('../../data/state-areas.csv')

abb = pd.read_csv('../../data/state-abbrevs.csv')

In [64]:
pop.shape

(2544, 4)

In [65]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [70]:
areas.shape

(52, 2)

In [69]:
abb.shape

(51, 2)

合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

In [71]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [72]:
abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [73]:
display(pop.shape,abb.shape)

(2544, 4)

(51, 2)

In [78]:
#此时的场景 left == outer left数据大于abb
#left效果比outer差一些
#abb 河北
pop_m = pop.merge(abb,left_on='state/region',right_on='abbreviation',how = 'outer')
pop_m.shape

(2544, 6)

去除abbreviation的那一列（axis=1）

In [79]:
pop_m.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [83]:
pop_m.drop('abbreviation',axis = 1,inplace=True)

ValueError: labels ['abbreviation'] not contained in axis

In [82]:
pop_m.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [88]:
pop_m.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [ ]:
#population 和 state这两列有数据缺失的情况

查看缺失数据

In [92]:
#为空的行索引
pop_m.loc[pop_m.isnull().any(axis = 1)]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


根据数据是否缺失情况显示数据，如果缺失为True，那么显示

找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

In [94]:
condition = pop_m['state'].isnull()
pop_m['state/region'][condition].unique()

array(['PR', 'USA'], dtype=object)

In [95]:
areas

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [ ]:
只有两个州，对应的州名为空

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

In [96]:
#Puerto Rico 

conditon = pop_m['state/region'] == 'PR'
condition


0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2514     True
2515     True
2516     True
2517     True
2518     True
2519     True
2520     True
2521     True
2522     True
2523     True
2524     True
2525     True
2526     True
2527     True
2528     True
2529     True
2530     True
2531     True
2532     True
2533     True
2534     True
2535     True
2536     True
2537     True
2538     True
2539     True
2540     True
2541     True
2542     True
2543     True
Name: state, Length: 2544, dtype: bool

In [97]:
pop_m['state'][condition] = 'Puerto Rico'

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
condition = pop_m['state/region'] == 'USA'
pop_m['state'][condition] = 'United State'

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [100]:
#刚才的填补操作，起作用了
pop_m.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

合并各州面积数据areas，使用左合并。

思考一下为什么使用外合并？



In [102]:
pop.head()
#人口的DataFrame和abb合并，有了州名全程
#可以和areas DataFrame进行合并

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [103]:
pop_areas_m = pop_m.merge(areas,how = 'outer')

继续寻找存在缺失数据的列

In [105]:
pop_areas_m.shape

(2544, 6)

In [109]:
areas

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [106]:
pop_areas_m.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

In [110]:
cond = pop_areas_m['area (sq. mi)'].isnull()
cond

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2514     True
2515     True
2516     True
2517     True
2518     True
2519     True
2520     True
2521     True
2522     True
2523     True
2524     True
2525     True
2526     True
2527     True
2528     True
2529     True
2530     True
2531     True
2532     True
2533     True
2534     True
2535     True
2536     True
2537     True
2538     True
2539     True
2540     True
2541     True
2542     True
2543     True
Name: area (sq. mi), Length: 2544, dtype: bool

In [111]:
pop_areas_m['state/region'][cond]

2496    USA
2497    USA
2498    USA
2499    USA
2500    USA
2501    USA
2502    USA
2503    USA
2504    USA
2505    USA
2506    USA
2507    USA
2508    USA
2509    USA
2510    USA
2511    USA
2512    USA
2513    USA
2514    USA
2515    USA
2516    USA
2517    USA
2518    USA
2519    USA
2520    USA
2521    USA
2522    USA
2523    USA
2524    USA
2525    USA
2526    USA
2527    USA
2528    USA
2529    USA
2530    USA
2531    USA
2532    USA
2533    USA
2534    USA
2535    USA
2536    USA
2537    USA
2538    USA
2539    USA
2540    USA
2541    USA
2542    USA
2543    USA
Name: state/region, dtype: object

去除含有缺失数据的行

In [112]:
pop_areas_m.shape

(2544, 6)

In [114]:
pop_areas_r = pop_areas_m.dropna()

In [115]:
pop_areas_r.shape

(2476, 6)

查看数据是否缺失

In [116]:
pop_areas_r.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

找出2010年的全民人口数据,df.query(查询语句)

In [117]:
pop_areas_r.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [120]:
t_2010 = pop_areas_r.query("ages == 'total' and year == 2010")

In [121]:
t_2010.shape

(52, 6)

In [122]:
t_2010

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0
283,CO,total,2010,5048196.0,Colorado,104100.0
293,CT,total,2010,3579210.0,Connecticut,5544.0
379,DE,total,2010,899711.0,Delaware,1954.0
389,DC,total,2010,605125.0,District of Columbia,68.0
475,FL,total,2010,18846054.0,Florida,65758.0


对查询结果进行处理，以state列作为新的行索引:set_index

In [124]:
t_2010.set_index('state',inplace=True)

In [126]:
t_2010

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0
Colorado,CO,total,2010,5048196.0,104100.0
Connecticut,CT,total,2010,3579210.0,5544.0
Delaware,DE,total,2010,899711.0,1954.0
District of Columbia,DC,total,2010,605125.0,68.0


计算人口密度。注意是Series/Series，其结果还是一个Series。

In [127]:
pop_density = t_2010['population']/t_2010["area (sq. mi)"]
pop_density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

排序，并找出人口密度最高的五个州sort_values()

In [128]:
type(pop_density)

pandas.core.series.Series

In [130]:
pop_density.sort_values(inplace=True)

找出人口密度最低的五个州

In [131]:
pop_density[:5]

state
Alaska           1.087509
Wyoming          5.768079
Montana          6.736171
North Dakota     9.537565
South Dakota    10.583512
dtype: float64

In [132]:
pop_density.tail()

state
Connecticut              645.600649
Rhode Island             681.339159
New Jersey              1009.253268
Puerto Rico             1058.665149
District of Columbia    8898.897059
dtype: float64

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算